In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set();
import scipy.stats as sps
import random

Импорт и препроцессинг данных

In [ ]:
# выгружаем датасет
from sklearn.datasets import fetch_openml
mnist = fetch_openml(data_id=554) # https://www.openml.org/d/554
# генерируем сегментирующую случайную переменую
rn = pd.Series(sps.randint.rvs(1, 101, size = len(mnist.data), random_state = 42))
# разбиваем на трейн/валидацию/тест
X = mnist.data
y = mnist.target
train_mask, val_mask, test_mask = (rn <= 60), ((rn > 60) & (rn <= 70)), (rn > 70)
X_train, y_train, X_test, y_test, X_val, y_val = X[train_mask], y[train_mask], X[test_mask], y[test_mask], X[val_mask], y[val_mask]

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

####Бинаризуем таргет

In [ ]:
def bin_repr(num):
  """
  функция принимает целое число и возвращает его двоичное представление в нужном нам формате  
  """
  bin_num = bin(num)[2:]
  bin_num = (4 - len(bin_num)) * '0' + bin_num
  return(bin_num)

In [ ]:
def make_binary_predictors(y):
    """
    Функция принимает Series y c категориальной переменной и возвращает DataFrame с бинарным представлением таргета и со столбцами из 0 и 1 для соответствующих позиций битов
    """
    bin_numbers = {k:v for k, v in zip([str(i) for i in range(10)], [bin_repr(i) for i in range(10)])}
    
    targets = pd.DataFrame()
    targets['target_bin'] = y.map(bin_numbers)
    targets['target_1_bit'] = targets['target_bin'].apply(lambda x: int(x[0]))
    targets['target_2_bit'] = targets['target_bin'].apply(lambda x: int(x[1]))
    targets['target_3_bit'] = targets['target_bin'].apply(lambda x: int(x[2]))
    targets['target_4_bit'] = targets['target_bin'].apply(lambda x: int(x[3]))

    return(targets)

####Подбор гиперпараметров

In [ ]:
params_dict = {
              'C': [0.5, 1, 5],
              'solver' : ['lbfgs', 'newton-cg']
              }

In [ ]:
from sklearn.model_selection import ParameterGrid
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


def grid_search(X_train,
                X_val,
                y_train,
                y_val,
                params_dict):
    '''
    Функция подбирает гиперпараметры мультиномиальной логитиеской регрессии для получения максимального значения accuracy
    на валидационной выборке, принимает:
    X_train -- DataFrame независимых переменных на обучающей выборке
    X_val -- DataFrame независимых переменных на валидационной выборке
    y_train -- Series таргета на обучающей выборке
    y_val -- Series таргета на валидационной выборке
    params_dict -- словарь гиперпараметров в формате {'paramater_nm':[value_1, value_2, ...]}
    '''

    best_acc = 0

    keys = list(params_dict.keys())

    best_params = dict.fromkeys(keys)

    param_list = list(ParameterGrid(params_dict))

    for cand in param_list:
      acc = 0
      clf = LogisticRegression(
          penalty=cand['penalty'] if ('penalty' in cand) else 'l2',
          dual=cand['dual'] if ('dual' in cand) else False,
          tol=cand['tol'] if ('tol' in cand) else 1e-4,         
          C=cand['C'] if ('C' in cand) else 1.0,
          fit_intercept=cand['fit_intercept'] if ('fit_intercept' in cand) else True,
          intercept_scaling=cand['intercept_scaling'] if ('intercept_scaling' in cand) else 1,
          class_weight=cand['class_weight'] if ('class_weight' in cand) else None,
          random_state=cand['random_state'] if ('random_state' in cand) else None,
          solver=cand['solver'] if ('solver' in cand) else 'lbfgs',
          max_iter=cand['max_iter'] if ('max_iter' in cand) else 1000,
          multi_class=cand['multinomial'] if ('multinomial' in cand) else 'auto',
          verbose=cand['verbose'] if ('verbose' in cand) else 0,
          warm_start=cand['warm_start'] if ('warm_start' in cand) else False,
          n_jobs=cand['n_jobs'] if ('n_jobs' in cand) else None,
          l1_ratio=cand['l1_ratio'] if ('l1_ratio' in cand) else None,
          )
      clf.fit(X_train, y_train)
      pred = clf.predict(X_val)
      acc = accuracy_score(y_val, pred)
      if acc > best_acc:
        best_acc = acc
        best_params = cand

    return(best_params)


grid_search(X_train, X_val, y_train, y_val, params_dict)

In [ ]:
from sklearn.metrics import roc_auc_score

#дефолтный логрес
clf_1 = LogisticRegression(max_iter=3000)
clf_1.fit(X_train, y_train)
y_pred_1 = clf_1.predict(X_test)
y_proba_1 = clf_1.predict_proba(X_test)
acc_1 = accuracy_score(y_test, y_pred_1)
auc_roc_1 = roc_auc_score(y_test, y_proba_1, average='macro', multi_class = 'ovr')

In [ ]:
#логрег с подобранными солвером и коэффициентом инверсии силы регуляризации 
clf_2 = LogisticRegression(C=0.5, solver='lbfgs', max_iter=3000)
clf_2.fit(X_train, y_train)
y_pred_2 = clf_2.predict(X_test)
y_proba_2 = clf_2.predict_proba(X_test)
acc_2 = accuracy_score(y_test, y_pred_2)
auc_roc_2 = roc_auc_score(y_test, y_proba_2, average='macro', multi_class = 'ovr')

####Бинарный классификатор

In [ ]:
class BinarisedTargetClassifier():
    '''
    класс BinarisedTargetClassifier -- мультиклассовый классификатор на основании нескольких бинарных логистических регрессий
    '''
    def __init__(self, fit_intercept=True, max_iter=3000):

        self.clf_1 = LogisticRegression(fit_intercept=fit_intercept, max_iter=max_iter)
        self.clf_2 = LogisticRegression(fit_intercept=fit_intercept, max_iter=max_iter)
        self.clf_3 = LogisticRegression(fit_intercept=fit_intercept, max_iter=max_iter)
        self.clf_4 = LogisticRegression(fit_intercept=fit_intercept, max_iter=max_iter)

    def fit(self, X_train, y_train):

        df_train = pd.DataFrame()
        df_train = make_binary_predictors(y_train)

        self.clf_1.fit(X_train, df_train['target_1_bit'])
        self.clf_2.fit(X_train, df_train['target_2_bit'])
        self.clf_3.fit(X_train, df_train['target_3_bit'])
        self.clf_4.fit(X_train, df_train['target_4_bit'])

    def predict(self, X):
        
        #получаем предсказания для каждого бита 
        pred_1 = self.clf_1.predict(X)
        pred_2 = self.clf_2.predict(X)
        pred_3 = self.clf_3.predict(X)
        pred_4 = self.clf_4.predict(X)

        #объединим в двоичное число
        size = pred_1.size

        pred_bit = list()
        for i in range(size):
          pred_bit.append(str(pred_1[i]) + str(pred_2[i]) + str(pred_3[i]) + str(pred_4[i]))

        #конвертируем в десятичное
        pred = pd.Series([int('0b' + i, 2) for i in pred_bit])

        """
        по факту у нас область значений предсказаний - целые числа от 0 до 15
        я действовал по следующей логике: если предсказано число > 9, то оно
        имеем вид: 1***, то есть модель, вероятно, хотела предсказать 8 или 9,
        тогда я смотрю на последний бит, если он равен 0, то меняю число на 8
        (8 = 1000), если равен 1, то меняю на 9 (9 = 1001), соответсвенно 
        заменяю по следующему правилу: 10, 12, 14 -> 8; 11, 13, 15 -> 9
        """

        pred_transform = pred.apply(lambda x : (8 if (x % 2 == 0) else 9) if x  > 9 else x)

        return pred_transform

    #перемножаем вероятности для каждого бита
    def predict_score(self, X): 
        proba_1 = self.clf_1.predict_proba(X)
        proba_2 = self.clf_2.predict_proba(X)
        proba_3 = self.clf_3.predict_proba(X)
        proba_4 = self.clf_4.predict_proba(X)

        proba_score = list()
        for i, j, k, l in zip(proba_1, proba_2, proba_3, proba_4):
          proba_element = [1] * 10
          i_ind, j_ind, k_ind, l_ind = np.argmax(i), np.argmax(j), np.argmax(k), np.argmax(l)
          # 1 бит
          for n in range(8):
              proba_element[n] = proba_element[n] * i[0]
          for n in range(8, 10):
              proba_element[n] = proba_element[n] * i[1]

          # 2 бит
          for n in list(range(4)) + list(range(8, 10)):
              proba_element[n] = proba_element[n] * j[0]
          for n in range(4, 8):
              proba_element[n] = proba_element[n] * j[1]
          # 3 бит
          for n in list(range(2)) + list(range(4, 6)) + list(range(8, 10)):
              proba_element[n] = proba_element[n] * k[0]
          for n in list(range(2, 4)) + list(range(6, 8)):
              proba_element[n] = proba_element[n] * k[1]
          # 4 бит
          for n in [0, 2, 4, 6, 8]:
              proba_element[n] = proba_element[n] * l[0]
          for n in [1, 3, 5, 7, 9]:
              proba_element[n] = proba_element[n] * l[1]
          sum_proba = sum(proba_element)
          probability = [m / sum_proba for m in proba_element]
          proba_score.append(probability)
        return proba_score

In [ ]:
clf_bin = BinarisedTargetClassifier()

#обучаем
clf_bin.fit(X_train, y_train)

In [ ]:
#предсказываем класс, считаем accuracy

y_test_int = pd.Series()
y_test_int = y_test.apply(lambda x: int(x))

y_pred = clf_bin.predict(X_test)
accuracy_bin = accuracy_score(y_test_int, y_pred)

#предсказываем вероятность, считаем AUC
score_bin = clf_bin.predict_score(X_test)
auc_bin = roc_auc_score(y_test_int, score_bin, average='macro', multi_class = 'ovr')

In [ ]:
res = pd.DataFrame(
{'regression_type' : ['defolt_logreg', 'logreg_after_grid_search', 'bin_logreg'],
'accuracy' : [acc_1, acc_2, accuracy_bin],
'macro AUC' : [auc_roc_1, auc_roc_2, auc_bin]
}
)
res

,regression_type,accuracy,macro AUC
0,defolt_logreg,0.912975,0.990129
1,logreg_after_grid_search,0.914741,0.990610
2,bin_logreg,0.725606,0.950212
